In [1]:
import json
import pandas as pd
import os
from psycopg2 import connect
import plotly_express as px

In [5]:
#transaction_aggregated

path="c:/Users/Hp/Desktop/Phonepe pulse/pulse/data/aggregated/transaction/country/india/state/"
state_list=os.listdir(path)
data={'State':[],'Quarter':[],'Transaction_type':[],'Transaction_amount':[],'Transaction_count':[],'Year':[]}

for i in state_list:
    path_i=path+i+'/'
    year_data=os.listdir(path_i)
    for j in year_data:
        path_j=path_i+j+'/'
        quarter_data=os.listdir(path_j)
        for k in quarter_data:
            path_k=path_j+k
            d=open(path_k,'r')
            data1=json.load(d)
            for z in data1["data"]["transactionData"]:
                Name=z['name']
                Count=z["paymentInstruments"][0]['count']
                Total_amount=z["paymentInstruments"][0]['amount']
                data['State'].append(i)
                data['Year'].append(j)
                data['Quarter'].append(int(k.strip('.json')))
                data['Transaction_type'].append(Name)
                data['Transaction_amount'].append(Total_amount)
                data['Transaction_count'].append(Count)
data_frame=pd.DataFrame(data)
#data_cleaning
data_frame['State']=data_frame['State'].str.replace("andaman-&-nicobar-islands","Andaman and Nicobar")
data_frame['State']=data_frame['State'].str.replace("-"," ")
data_frame['State']=data_frame['State'].str.replace("dadra & nagar haveli & daman & diu","dadra and nagar haveli and daman and diu")
data_frame['State']=data_frame['State'].str.title()
data_frame['Transaction_type']=data_frame['Transaction_type'].str.replace("Recharge & bill payments","Recharge and bill payments")
data_frame['Transaction_type']=data_frame['Transaction_type'].str.replace("-"," ")

#SQL section:
myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()
create_table='''create table if not exists agg_transaction(state varchar(100),
                                                        quarter int,
                                                        transaction_type varchar(50),
                                                        transaction_amount bigint,
                                                        transaction_count bigint,
                                                        year int)'''
cursor.execute(create_table)
myconnection.commit()
for index,row in data_frame.iterrows():
    insert_table='''insert into agg_transaction(state,
                                                quarter,
                                                transaction_type,
                                                transaction_amount,
                                                transaction_count,
                                                year)values(%s,%s,%s,%s,%s,%s)'''
    values=(row['State'],
            row['Quarter'],
            row['Transaction_type'],
            row['Transaction_amount'],
            row['Transaction_count'],
            row['Year'])
    cursor.execute(insert_table,values)
    myconnection.commit()


In [6]:
#user_aggregrated
users_path="c:/Users/Hp/Desktop/Phonepe pulse/pulse/data/aggregated/user/country/india/state/"
users_state=os.listdir(users_path)
users_data={'Brand':[],'Total_count':[],'Percentage':[],'Quarter':[],'State':[],'Year':[]}

for x in users_state:
    path_x=users_path+x+'/'
    users_year=os.listdir(path_x)
    for y in users_year:
        path_y=path_x+y+'/'
        users_quarter=os.listdir(path_y)
        for z in users_quarter:
            path_z=path_y+z
            d1=open(path_z,'r')
            data2=json.load(d1)
            try:
                for w in data2["data"]["usersByDevice"]:
                    Device_brand=w['brand']
                    count=w["count"]
                    percentage=w["percentage"]
                    users_data['Brand'].append(Device_brand)
                    users_data['Total_count'].append(count)
                    users_data['Percentage'].append(percentage)
                    users_data['Quarter'].append(int(z.strip('.json')))
                    users_data['State'].append(x)
                    users_data['Year'].append(y)
            except:
                pass
data_frame1=pd.DataFrame(users_data)

data_frame1['State']=data_frame1['State'].str.replace("andaman-&-nicobar-islands","Andaman and Nicobar")
data_frame1['State']=data_frame1['State'].str.replace("-"," ")
data_frame1['State']=data_frame1['State'].str.replace("dadra & nagar haveli & daman & diu","dadra and nagar haveli and daman and diu")
data_frame1['State']=data_frame1['State'].str.title()

myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()
create_table='''create table if not exists agg_user(brand varchar(100),
                                                        total_count int,
                                                        percentage varchar(50),
                                                        quarter int,
                                                        state varchar(50),
                                                        year int)'''
cursor.execute(create_table)
myconnection.commit()
for index,row in data_frame1.iterrows():
    insert_table='''insert into agg_user(brand,
                                        total_count,
                                        percentage,
                                        quarter,
                                        state,
                                        year)values(%s,%s,%s,%s,%s,%s)'''
    values=(row['Brand'],
            row['Total_count'],
            row['Percentage'],
            row['Quarter'],
            row['State'],
            row['Year'])
    cursor.execute(insert_table,values)
    myconnection.commit()



In [7]:
#map_transaction
map_path="C:/Users/Hp/Desktop/Phonepe pulse/pulse/data/map/transaction/hover/country/india/state/"
map_list=os.listdir(map_path)
map_data={'City_name':[],'State':[],'Transaction_count':[],'Total_amount':[],'Year':[],'Quarter':[]}
for a in map_list:
    path_a=map_path+a+'/'
    map_year=os.listdir(path_a)
    for b in map_year:
        path_b=path_a+b+'/'
        map_quarter=os.listdir(path_b)
        for c in map_quarter:
            path_c=path_b+c
            data3=open(path_c,'r')
            d3=json.load(data3)
            for d in d3["data"]["hoverDataList"]:
                map_name=d["name"]
                t_count=d["metric"][0]["count"]
                t_amount=d["metric"][0]["amount"]
                map_data["City_name"].append(map_name)
                map_data["Transaction_count"].append(t_count)
                map_data["Total_amount"].append(t_amount)
                map_data["Quarter"].append(int(c.strip(".json")))
                map_data['Year'].append(b)
                map_data['State'].append(a)
data_frame2=pd.DataFrame(map_data)

data_frame2['State']=data_frame2['State'].str.replace("andaman-&-nicobar-islands","Andaman and Nicobar")
data_frame2['State']=data_frame2['State'].str.replace("-"," ")
data_frame2['State']=data_frame2['State'].str.replace("dadra & nagar haveli & daman & diu","dadra and nagar haveli and daman and diu")
data_frame2['State']=data_frame2['State'].str.title()

myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()
create_table='''create table if not exists map_transaction(city varchar(200),
                                                        transaction_count bigint,
                                                        total_amount bigint,
                                                        quarter int,
                                                        year int,
                                                        state varchar(50))'''
cursor.execute(create_table)
myconnection.commit()
for index,row in data_frame2.iterrows():
    insert_table='''insert into map_transaction(city,
                                        transaction_count,
                                        total_amount,
                                        quarter,
                                        year,
                                        state)values(%s,%s,%s,%s,%s,%s)'''
    values=(row['City_name'],
            row['Transaction_count'],
            row['Total_amount'],
            row['Quarter'],
            row['Year'],
            row['State'])
    cursor.execute(insert_table,values)
    myconnection.commit()


In [8]:
#map_user

mapuser_path="C:/Users/Hp/Desktop/Phonepe pulse/pulse/data/map/user/hover/country/india/state/"
map_state=os.listdir(mapuser_path)
users_map={'District':[],'State':[],'year':[],'quarter':[],'Registered_users':[],'App_opens':[]}
for state in map_state:
    state_path=mapuser_path+state+'/'
    states=os.listdir(state_path)
    for year in states:
        y_path=state_path+year+'/'
        years=os.listdir(y_path)
        for quarter in years:
            q_path=y_path+quarter
            data4=open(q_path,'r')
            d4=json.load(data4)
            for i in d4['data']['hoverData'].items():
                district_name=i[0]
                r_users=i[1]["registeredUsers"]
                a_opens=i[1]['appOpens']
                users_map['District'].append(district_name)
                users_map['Registered_users'].append(r_users)
                users_map['App_opens'].append(a_opens)
                users_map['year'].append(year)
                users_map['State'].append(state)
                users_map['quarter'].append(int(quarter.strip(".json")))
data_frame3=pd.DataFrame(users_map)

data_frame3['State']=data_frame3['State'].str.replace("andaman-&-nicobar-islands","Andaman and Nicobar")
data_frame3['State']=data_frame3['State'].str.replace("-"," ")
data_frame3['State']=data_frame3['State'].str.replace("dadra & nagar haveli & daman & diu","dadra and nagar haveli and daman and diu")
data_frame3['State']=data_frame3['State'].str.title()

myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()
create_table='''create table if not exists map_user(district varchar(200),
                                                        registered_users bigint,
                                                        app_opens bigint,
                                                        quarter int,
                                                        year int,
                                                        state varchar(50))'''
cursor.execute(create_table)
myconnection.commit()
for index,row in data_frame3.iterrows():
    insert_table='''insert into map_user(district,
                                        registered_users,
                                        app_opens,
                                        quarter,
                                        year,
                                        state)values(%s,%s,%s,%s,%s,%s)'''
    values=(row['District'],
            row['Registered_users'],
            row['App_opens'],
            row['quarter'],
            row['year'],
            row['State'])
    cursor.execute(insert_table,values)
    myconnection.commit()



In [9]:
#top_transaction

top_path="c:/Users/Hp/Desktop/Phonepe pulse/pulse/data/top/transaction/country/india/state/"
top_state=os.listdir(path)
top_trans={'Year':[],'State':[],'Quarter':[],'Transaction_type':[],'Transaction_amount':[],'Transaction_count':[]}

for i in state_list:
    path_i=path+i+'/'
    year_data=os.listdir(path_i)
    for j in year_data:
        path_j=path_i+j+'/'
        quarter_data=os.listdir(path_j)
        for k in quarter_data:
            path_k=path_j+k
            d=open(path_k,'r')
            data1=json.load(d)
            for z in data1["data"]["transactionData"]:
                Name=z['name']
                Count=z["paymentInstruments"][0]['count']
                Total_amount=z["paymentInstruments"][0]['amount']
                top_trans['Year'].append(j)
                top_trans['State'].append(i)
                top_trans['Quarter'].append(int(k.strip('.json')))
                top_trans['Transaction_type'].append(Name)
                top_trans['Transaction_amount'].append(Total_amount)
                top_trans['Transaction_count'].append(Count)
data_frame4=pd.DataFrame(top_trans)
data_frame4['State']=data_frame4['State'].str.replace("andaman-&-nicobar-islands","Andaman and Nicobar")
data_frame4['State']=data_frame4['State'].str.replace("-"," ")
data_frame4['State']=data_frame4['State'].str.replace("dadra & nagar haveli & daman & diu","dadra and nagar haveli and daman and diu")
data_frame4['State']=data_frame4['State'].str.title()

myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()
create_table='''create table if not exists top_transaction(state varchar(200),
                                                        transaction_type varchar(200),
                                                        transaction_count bigint,
                                                        transaction_amount bigint,
                                                        quarter int,
                                                        year int)'''
cursor.execute(create_table)
myconnection.commit()
for index,row in data_frame4.iterrows():
    insert_table='''insert into top_transaction(state,
                                        transaction_type,
                                        transaction_count,
                                        transaction_amount,
                                        quarter,
                                        year)values(%s,%s,%s,%s,%s,%s)'''
    values=(row['State'],
            row['Transaction_type'],
            row['Transaction_count'],
            row['Transaction_amount'],
            row['Quarter'],
            row['Year'])
    cursor.execute(insert_table,values)
    myconnection.commit()


In [10]:
#top_user
mapuser_path="C:/Users/Hp/Desktop/Phonepe pulse/pulse/data/top/user/country/india/state/"
map_state=os.listdir(mapuser_path)
users_map={'District':[],'State':[],'year':[],'quarter':[],'Registered_users':[]}
for state in map_state:
    state_path=mapuser_path+state+'/'
    states=os.listdir(state_path)
    for year in states:
        y_path=state_path+year+'/'
        years=os.listdir(y_path)
        for quarter in years:
            q_path=y_path+quarter
            data4=open(q_path,'r')
            d4=json.load(data4)
            for i in d4['data']['districts']:
                district_name=i['name']
                r_users=i["registeredUsers"]
                users_map['District'].append(district_name)
                users_map['Registered_users'].append(r_users)
                users_map['year'].append(year)
                users_map['State'].append(state)
                users_map['quarter'].append(int(quarter.strip(".json")))
data_frame5=pd.DataFrame(users_map)
data_frame5['State']=data_frame5['State'].str.replace("andaman-&-nicobar-islands","Andaman and Nicobar")
data_frame5['State']=data_frame5['State'].str.replace("-"," ")
data_frame5['State']=data_frame5['State'].str.replace("dadra & nagar haveli & daman & diu","dadra and nagar haveli and daman and diu")
data_frame5['State']=data_frame5['State'].str.title()
data_frame5['District']=data_frame5['District'].str.title()

myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()
create_table='''create table if not exists top_user(district varchar(200),
                                                        registered_users bigint,
                                                        quarter int,
                                                        year int,
                                                        state varchar(50))'''
cursor.execute(create_table)
myconnection.commit()
for index,row in data_frame5.iterrows():
    insert_table='''insert into top_user(district,
                                        registered_users,
                                        quarter,
                                        year,
                                        state)values(%s,%s,%s,%s,%s)'''
    values=(row['District'],
            row['Registered_users'],
            row['quarter'],
            row['year'],
            row['State'])
    cursor.execute(insert_table,values)
    myconnection.commit()

In [38]:
myconnection=connect(host='127.0.0.1',user='postgres',password='yusuff@12345',port='5432',database='phonepe')
cursor=myconnection.cursor()

cursor.execute("SELECT * FROM agg_transaction")
myconnection.commit()
table1=cursor.fetchall()
agg_trans=pd.DataFrame(table1,columns=["State","Quarter","Transaction Type","Transaction amount","Transaction count","Year"])

cursor.execute("SELECT * FROM agg_user")
myconnection.commit()
table1=cursor.fetchall()
agg_user=pd.DataFrame(table1,columns=["Mobile Brand","Total count","Percentage","Quarter","State"])

cursor.execute("SELECT * FROM map_transaction")
myconnection.commit()
table1=cursor.fetchall()
map_trans=pd.DataFrame(table1,columns=["City","Transaction count","Total amount","quarter","Year","State"])

cursor.execute("SELECT * FROM map_user")
myconnection.commit()
table1=cursor.fetchall()
map_user=pd.DataFrame(table1,columns=["District","Registered Users","App Opens","Quarter","Year","State"])

cursor.execute("SELECT * FROM top_transaction")
myconnection.commit()
table1=cursor.fetchall()
top_trans=pd.DataFrame(table1,columns=["State","Transaction Type","Transaction count","Transaction amount","Quarter","Year"])

cursor.execute("SELECT * FROM top_user")
myconnection.commit()
table1=cursor.fetchall()
top_user=pd.DataFrame(table1,columns=["District","Registered Users","Quarter","Year","State"])

In [64]:
def trans_amount_count(year):
    trans_y=agg_trans[agg_trans['Year']==year]
    trans_y.reset_index(drop=True,inplace=True)
    trans_g=trans_y.groupby("State")[['Transaction count','Transaction amount']].sum()
    trans_g.reset_index(inplace=True)
    figure_trans_count=px.bar(trans_g, x="State" , y="Transaction count",title="Transaction count",height=600,width=750,color_discrete_sequence=px.colors.sequential.Aggrnyl)
    figure_trans_amount=px.bar(trans_g,x='State',y="Transaction amount",height=600,width=750,color_discrete_sequence=px.colors.sequential.Bluered_r)

In [65]:
figure_trans_count

In [66]:
figure_trans_amount
